##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrando seu código TFLite para TF2

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tflite"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/tflite.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

O [TensorFlow Lite](https://www.tensorflow.org/lite/guide) (TFLite) é um conjunto de ferramentas que ajuda os desenvolvedores a executar a inferência de aprendizado de máquina em um dispositivo (dispositivos móveis, embarcados e IoT). O [conversor TFLite](https://www.tensorflow.org/lite/convert) é uma dessas ferramentas que converte modelos TF existentes num formato de modelo TFLite otimizado que pode ser executado com eficiência no dispositivo.

Neste documento, você aprenderá quais alterações precisa fazer no seu código de conversão de TF para TFLite, e verá alguns exemplos que fazem a mesma coisa.

## Alterações no seu código de conversão de TF para TFLite

- Se você estiver usando um formato de modelo TF1 legado (como um arquivo Keras, frozen GraphDef, checkpoints, tf.Session), atualize-o para TF1/TF2 SavedModel e use a API do conversor TF2 `tf.lite.TFLiteConverter.from_saved_model(...)` para convertê-lo num modelo TFLite (consulte a Tabela 1).

- Atualize os sinalizadores da API do conversor (consulte a Tabela 2).

- Remova as APIs legadas, como `tf.lite.constants`. (por exemplo: substitua `tf.lite.constants.INT8` por `tf.int8`)

// Tabela 1 // Atualização da API TFLite Python Converter

API TF1 | API TF2
--- | ---
`tf.lite.TFLiteConverter.from_saved_model('saved_model/',..)` | *suportado*
`tf.lite.TFLiteConverter.from_keras_model_file('model.h5',..)` | *removido (atualize para o formato SavedModel)*
`tf.lite.TFLiteConverter.from_frozen_graph('model.pb',..)` | *removido (atualize para o formato SavedModel)*
`tf.lite.TFLiteConverter.from_session(sess,...)` | *removido (atualize para o formato SavedModel)*

&lt;style&gt;   .table {margin-left: 0 !important;} &lt;/style&gt;

// Tabela 2 // Atualização de sinalizadores da API do TFLite Python Converter

API TF1 | API TF2
--- | ---
`allow_custom_ops`<br>`optimizations`<br> `representative_dataset`<br>`target_spec` <br>`inference_input_type`<br>`inference_output_type`<br>`experimental_new_converter`<br> `experimental_new_quantizer` | *suportado* <br><br><br><br><br><br><br><br>
`input_tensors`<br>`output_tensors`<br>`input_arrays_with_shape`<br>`output_arrays`<br>`experimental_debug_info_func` | *removido (argumentos da API do conversor não suportados)*<br><br><br><br><br>
`change_concat_input_ranges`<br>`default_ranges_stats`<br>`get_input_arrays()`<br>`inference_type`<br>`quantized_input_stats`<br> `reorder_across_fake_quant` | *removido (workflows de quantização não suportados)*<br><br><br><br><br><br>
`conversion_summary_dir`<br>`dump_graphviz_dir`<br>`dump_graphviz_video` | *removido (visualize os modelos usando [Netron](https://lutzroeder.github.io/netron/) ou [visualize.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/tools/visualize.py))*<br><br><br>
`output_format`<br>`drop_control_dependency` | *removido (recursos não suportados no TF2)*<br><br>

## Exemplos

Agora você percorrerá alguns exemplos para converter modelos TF1 legados para SavedModels do TF1/TF2 e, em seguida, convertê-los em modelos TF2 TFLite.

### Configuração

Comece com os imports necessários do TensorFlow.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

import shutil
def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass

Crie todos os formatos de modelo TF1 necessários.

In [ ]:
# Create a TF1 SavedModel
SAVED_MODEL_DIR = "tf_saved_model/"
remove_dir(SAVED_MODEL_DIR)
with tf1.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=(3,), name='input')
    output = input + 2
    # print("result: ", sess.run(output, {input: [0., 2., 4.]}))
    tf1.saved_model.simple_save(
        sess, SAVED_MODEL_DIR,
        inputs={'input': input}, 
        outputs={'output': output})
print("TF1 SavedModel path: ", SAVED_MODEL_DIR)

# Create a TF1 Keras model
KERAS_MODEL_PATH = 'tf_keras_model.h5'
model = tf1.keras.models.Sequential([
    tf1.keras.layers.InputLayer(input_shape=(128, 128, 3,), name='input'),
    tf1.keras.layers.Dense(units=16, input_shape=(128, 128, 3,), activation='relu'),
    tf1.keras.layers.Dense(units=1, name='output')
])
model.save(KERAS_MODEL_PATH, save_format='h5')
print("TF1 Keras Model path: ", KERAS_MODEL_PATH)

# Create a TF1 frozen GraphDef model
GRAPH_DEF_MODEL_PATH = tf.keras.utils.get_file(
    'mobilenet_v1_0.25_128',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.25_128_frozen.tgz',
    untar=True,
) + '/frozen_graph.pb'

print("TF1 frozen GraphDef path: ", GRAPH_DEF_MODEL_PATH)

### 1. Converta um TF1 SavedModel para um modelo TFLite


#### Antes: Convertendo com TF1

Este é um código típico para conversão TFlite no estilo TF1.


In [ ]:
converter = tf1.lite.TFLiteConverter.from_saved_model(
    saved_model_dir=SAVED_MODEL_DIR,
    input_arrays=['input'],
    input_shapes={'input' : [3]}
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()
# Ignore warning: "Use '@tf.function' or '@defun' to decorate the function."

#### Depois: Convertendo com TF2

Converta diretamente o TF1 SavedModel para um modelo TFLite, com um conjunto menor de sinalizadores do conversor v2.

In [ ]:
# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir=SAVED_MODEL_DIR)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

### 2. Converta um arquivo de modelo TF1 Keras para um modelo TFLite

#### Antes: Convertendo com TF1

Este é um código típico para conversão TFlite no estilo TF1.

In [ ]:
converter = tf1.lite.TFLiteConverter.from_keras_model_file(model_file=KERAS_MODEL_PATH)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### Depois: Convertendo com TF2

Primeiro, converta o arquivo de modelo TF1 Keras para um TF2 SavedModel e, em seguida, converta-o para um modelo TFLite, com um conjunto menor de sinalizadores do conversor v2.

In [ ]:
# Convert TF1 Keras model file to TF2 SavedModel.
model = tf.keras.models.load_model(KERAS_MODEL_PATH)
model.save(filepath='saved_model_2/')

# Convert TF2 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_2/')
tflite_model = converter.convert()

### 3. Converta um frozen GraphDef TF1 para um modelo TFLite


#### Antes: Convertendo com TF1

Este é um código típico para conversão TFlite no estilo TF1.

In [ ]:
converter = tf1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=GRAPH_DEF_MODEL_PATH,
    input_arrays=['input'],
    input_shapes={'input' : [1, 128, 128, 3]},
    output_arrays=['MobilenetV1/Predictions/Softmax'],
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### Depois: Convertendo com TF2

Primeiro, converta o frozen GraphDef TF1 para um SavedModel TF1 e, em seguida, converta-o para um modelo TFLite, com um conjunto menor de sinalizadores do conversor v2.


In [ ]:
## Convert TF1 frozen Graph to TF1 SavedModel.

# Load the graph as a v1.GraphDef
import pathlib
gdef = tf.compat.v1.GraphDef()
gdef.ParseFromString(pathlib.Path(GRAPH_DEF_MODEL_PATH).read_bytes())

# Convert the GraphDef to a tf.Graph
with tf.Graph().as_default() as g:
  tf.graph_util.import_graph_def(gdef, name="")

# Look up the input and output tensors.
input_tensor = g.get_tensor_by_name('input:0') 
output_tensor = g.get_tensor_by_name('MobilenetV1/Predictions/Softmax:0')

# Save the graph as a TF1 Savedmodel
remove_dir('saved_model_3/')
with tf.compat.v1.Session(graph=g) as s:
  tf.compat.v1.saved_model.simple_save(
      session=s,
      export_dir='saved_model_3/',
      inputs={'input':input_tensor},
      outputs={'output':output_tensor})

# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_3/')
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

# Mais recursos

- Consulte o [Guia TFLite](https://www.tensorflow.org/lite/guide) para saber mais sobre os workflows e os recursos mais recentes.
- Se você estiver usando código TF1 ou formatos de modelo TF1 legados (arquivos Keras `.h5`, frozen GraphDef `.pb`, etc.), atualize seu código e migre seus modelos para o [formato TF2 SavedModel](https://www.tensorflow.org/guide/saved_model).
